In [ ]:
import pandas as pd

# Magics
%load_ext autoreload
%autoreload 2

# 1. Load Quotes

### Huge base

In [ ]:
corpus = pd.read_csv('quotes.csv', quotechar='"', quoting=0)
corpus = corpus.loc[corpus['author'].apply(lambda s: (type(s) is str and s[0].isupper()))]

In [ ]:
corpus['author'] = corpus['author'].apply(lambda s: s.replace(', Jr.', ' Jr.'))
corpus[['author','work']] = corpus['author'].str.split(',', n=1, expand=True)
corpus = corpus.loc[~corpus['author'].apply(lambda s: len(s) > 30)]

In [ ]:
corpus['author'].nunique()

In [ ]:
corpus['author'].value_counts()[:20]

### Clean, smaller base

In [ ]:
corpus_classical = pd.read_csv('quotes_all.csv', sep=';')

In [ ]:
corpus = corpus_classical.groupby(['quote'])[['author','category']].agg({'author':'first', 'category':list}).reset_index()

In [ ]:
corpus.loc[(corpus['author'] == 'Antoine de Saint-Exupéry')].head()

### Disambiguate author names

In [ ]:
# import scripts

# deambiguate_authors(corpus["authors"])

## Remove profanity

In [ ]:
from profanity_check import predict_prob

corpus = corpus.reset_index(drop=True).loc[
    ~(
        pd.Series(predict_prob(corpus['category'].astype('U').values) > 0.8)
        | pd.Series(predict_prob(corpus['category'].astype('U').values) > 0.8)
    )
]

In [ ]:
corpus.to_csv('clean_quotes.csv', sep='|')

# 2. Authors
### Filter out recent/unknown authors

In [ ]:
from tqdm import tqdm

authors = corpus["author"].unique()

results = []
deambiguate_names = {"Anonymous": "Anonymous"}
eliminated_authors = []

In [ ]:
# authors = ['Antoine de Rivarol', 'Antoine de Saint-Exupéry']

In [ ]:
from scripts import return_author_results

for author in (pbar:=tqdm(authors)):
    pbar.set_description(author)
    if author not in deambiguate_names and author not in eliminated_authors:
        author_results = None
        try:
            print("OKOK")
            author_results = return_author_results(author)
        except:
            print("fail", author)
        if author_results is None:
            eliminated_authors.append(author)
        else:
            name, description, extract_html, thumbnail_url = author_results
            deambiguate_names[author] = name
            results.append([name, description, extract_html, thumbnail_url])

In [ ]:
def turn_into_dict(row):
    return {
        "name": row[0],
        "description": row[1],
        "extract_html": row[2],
        "thumbnail_url": row[3],
    }

results_dicts = [turn_into_dict(row) for row in results]
results_df = pd.DataFrame(results_dicts)

In [ ]:
eliminated_authors = [i for i in eliminated_authors if i != "Queen Victoria" and i!= 'Maria Callas']

In [ ]:
results_df.to_csv('output/authors.csv', sep='|', index=False)

In [ ]:
corpus = corpus[corpus['author'].isin(good_old_authors)]

# Export

### Export authors table

In [ ]:
results_df = pd.read_csv('output/authors.csv', sep='|')

In [ ]:
results_df.to_csv('output/results_sep.csv', sep='|', index=False)

In [ ]:
corpus.to_csv('quotes_classical_clean.csv', sep='|', index=False)

### To SQL

In [ ]:
MAX_DESCRIPTION_LENGTH = 2500

from scripts import open_sql_connection

conn = open_sql_connection()
cursor = conn.cursor()

In [ ]:
sql = '''DROP TABLE authors;'''

cursor.execute(sql)

sql = f'''CREATE TABLE authors(name varchar(30),\
description varchar(150), extract_html varchar({MAX_DESCRIPTION_LENGTH}), thumbnail_url varchar(600));'''

cursor.execute(sql)

In [ ]:
import json
results_df['thumbnail_url'] = results_df['thumbnail_url'].apply(lambda s: (s if type(s) == str else ''))
# for col in ['thumbnail_url', 'description', 'name', 'extract_html']:
#     results_df[col] = results_df[col].apply(lambda s: s.replace("'", "`"))
results_df.iloc[1].to_dict()

In [ ]:
req = "INSERT INTO authors(name,description,extract_html,thumbnail_url) VALUES ('Anonymous', 'Someone wrote this, and it seemed important enough to note it.', '', 'https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Anonymous_emblem.svg/800px-Anonymous_emblem.svg.png')"
cursor.execute(req)

In [ ]:
from tqdm import tqdm
done = []
for i, row in tqdm(results_df.iterrows()):
    extract_html = row["extract_html"]
    if len(extract_html) > MAX_DESCRIPTION_LENGTH:
        extract_html = extract_html.split('.')[0] + '</p>'
    req = """INSERT INTO authors(name,description,extract_html,thumbnail_url)\
        VALUES (%(name)s, %(description)s, %(extract_html)s, %(thumbnail_url)s);"""
    if i not in done:
        try:
            cursor.execute(req, dict(
                name=row["name"],
                description=row["description"],
                extract_html=extract_html,
                thumbnail_url=row["thumbnail_url"],
            ))
            done.append(i)
        except:
            print("Error on row", row.to_dict())

In [ ]:
sql3 = '''select * from authors;'''
cursor.execute(sql3)
authors = []
for row in cursor.fetchall():
    authors.append(row[0])

conn.commit()
conn.close()

In [ ]:
[i for i in authors if 'Antoine' in i]

# 3. Filter quotes based on authors

In [ ]:
corpus = corpus[corpus["author"].isin(authors)]

In [ ]:
corpus = corpus.reset_index(drop=True).reset_index()

In [ ]:
corpus.to_csv('output/quotes_export.csv', sep='|', index=False)

In [ ]:
corpus.loc[(corpus['author'] == 'Antoine de Rivarol')].head()

### Export

In [ ]:
corpus = pd.read_csv('output/quotes_export.csv', sep='|')

In [ ]:
from scripts import open_sql_connection
conn = open_sql_connection()
cursor = conn.cursor()

In [ ]:
sql = '''DROP TABLE quotes;'''
cursor.execute(sql)

sql = '''CREATE TABLE quotes(index int, quote varchar(500), author varchar(50));'''
cursor.execute(sql)

In [ ]:
sql3 = '''select quote from quotes;'''
cursor.execute(sql3)
quotes = []
for row in cursor.fetchall():
    quotes.append(row[0])


In [ ]:
from tqdm import tqdm

for _, row in tqdm(corpus.iterrows()):
    if row["quote"] not in quotes:
        req = """INSERT INTO quotes(index, quote,author) VALUES (%(index)s, %(quote)s, %(author)s);"""
        cursor.execute(
            req,
            dict(
                index=row["index"], 
                quote=row["quote"], 
                author=row["author"]
            ),
        )

In [ ]:
conn.commit()
conn.close()

# End of notebook